In [1]:
from leagues.models import Competition,Season
from base_app.helpers import best_fuzzy_match
from games.models import Game
from datetime import datetime
import pandas as pd
import numpy as np
import os

In [2]:
EV_PATH = "D:\MetaData"
ST_PATH = "D:\MetaData-shot"

In [3]:
event_files = []
for base,_,files in os.walk(EV_PATH):
    for f in files:
        if 'shot' not in f :
            event_files.append(f"{base}\{f}")

shot_files = []
for base,_,files in os.walk(ST_PATH):
    for f in files:
        shot_files.append(f"{base}\{f}")

event_files = list(set(event_files))
shot_files = list(set(shot_files))


In [4]:
len(event_files),len(shot_files)

(547, 199)

In [11]:
map_example = []
non_mapped_season = []
for season in Season.objects.all():
    conf = season.competition.confederation
    region = season.competition.country
    
    e_name = season.competition.name_scoresaway.replace(' ','_').replace(".","")
    s_name = season.competition.name_fotmob.replace(' ','_').replace(".","")
    
    e_s_name = season.name.replace('/','_').replace(' ','_')
    s_s_name = season.name_fotmob.replace('/','_').replace(' - ','_')

    if season.competition_id == 27 :
        e_search_str = f"D:\\MetaData\\{conf}\\{region}\\{e_name}_{e_s_name}_events.xlsx"
        s_search_str = [f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{e_s_name}_Apertura_shots.xlsx",
                        f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{e_s_name}_Clausura_shots.xlsx"]
        if e_search_str in event_files and s_search_str[0] in shot_files and s_search_str[1] in shot_files  :
            map_example.append((season,e_search_str,"__||__".join(s_search_str)))
    else:
        e_search_str = f"D:\\MetaData\\{conf}\\{region}\\{e_name}_{e_s_name}_events.xlsx"
        s_search_str = f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{s_s_name}_shots.xlsx"

    if e_search_str in event_files and s_search_str in shot_files  :
        map_example.append((season,e_search_str,s_search_str))
    else:
        non_mapped_season.append((season,e_search_str))

In [12]:
len(map_example),len(non_mapped_season)

(199, 381)

In [13]:
from datetime import timedelta
                               # e_df,s_df,event_teams,shot_teams,mapping_dict,8,return_counts=True
def match_and_count_for_season(e_df,s_df,event_teams,shot_teams,hrs,return_counts=False):
    max_diff_hours = hrs
    results = []
    merged = []
    corr = 0
    wrong = 0
    ups = 0
    not_req = 0
    min_dt_of_event = s_df['datetime'].min()
    for i, row in e_df.iterrows():
        e_home = row['home_team']
        e_away = row['away_team']
        e_date = row['datetime']

        if (e_date-min_dt_of_event).total_seconds()/3600 < -8 :
            not_req+=1
            continue
        fuzzy_home = best_fuzzy_match(e_home, shot_teams,threshold_multi=75)
        fuzzy_away = best_fuzzy_match(e_away, shot_teams,threshold_multi=75)
        
        # Get top fuzzy keys (most probable names)
        possible_homes = list(fuzzy_home.keys())
        possible_aways = list(fuzzy_away.keys())
        
        # Check all possible home-away combinations in df
        mask = (
            (s_df['home_team'].isin(possible_homes)) &
            (s_df['away_team'].isin(possible_aways)) &
            (abs(s_df['datetime'] - e_date) <= timedelta(hours=max_diff_hours))
        )
        
        matches = s_df[mask]
        if matches.shape[0] >1:
            print("fakk, there are more ... do something kent saab")
            # print(f"\n{e_home} -> {fuzzy_home}")
            # print(f"{e_away} -> {fuzzy_away}")
            # print(f"datetime : {e_date} \n\n")
            
            # ups+=1
            fuzzy_home = best_fuzzy_match(e_home, shot_teams,threshold_multi=85)
            fuzzy_away = best_fuzzy_match(e_away, shot_teams,threshold_multi=85)
            
            # Get top fuzzy keys (most probable names)
            possible_homes = list(fuzzy_home.keys())
            possible_aways = list(fuzzy_away.keys())
            
            # Check all possible home-away combinations in df
            mask = (
                (s_df['home_team'].isin(possible_homes)) &
                (s_df['away_team'].isin(possible_aways)) &
                (abs(s_df['datetime'] - e_date) <= timedelta(hours=max_diff_hours))
            )
            
            matches = s_df[mask]
            if matches.shape[0] > 1 :
                print("Fadd lo bhai hun dooji waar!!!!!!!!!!")
                print(f"\n{e_home} -> {fuzzy_home}")
                print(f"{e_away} -> {fuzzy_away}")
                print(f"datetime : {e_date} \n\n")
                ups+=1
                
        # print(f"Found {matches.shape[0]} matched for the fuzzy map combinations")
        # print(f"\n{e_home} -> {fuzzy_home}")
        # print(f"{e_away} -> {fuzzy_away}")
        # print(f"datetime : {e_date}")
        if not matches.empty:
            correct_match = matches.iloc[0]
                    
            temp = {}
            temp['e_home_team'] = e_home
            temp['s_home_team'] = correct_match['home_team']
            
            temp['e_away_team'] = e_away
            temp['s_away_team'] = correct_match['away_team']
        
            temp['e_datetime'] = e_date
            temp['s_datetime'] = correct_match['datetime']
        
            temp['event_url'] = row['event_url']
            temp['shot_url'] = correct_match['shot_url']
            merged.append(temp) 
            corr+=1
        else:
            # print("\n Adding Blanks \n")
            temp = {}
            temp['e_home_team'] = e_home
            temp['s_home_team'] = "NA"
            
            temp['e_away_team'] = e_away
            temp['s_away_team'] = "NA"
        
            temp['e_datetime'] = e_date
            temp['s_datetime'] = "NA"
        
            temp['event_url'] = row['event_url']
            temp['shot_url'] = "NA"
            merged.append(temp) 
            if fuzzy_home == {} or fuzzy_away == {} :
            
                # print(row)
                # print()
                print(f"\nF : {e_home} -> {fuzzy_home}")
                print(f"{e_away} -> {fuzzy_away}")
                print(f"datetime : {e_date}")
                print(f"Diff : {e_date-min_dt_of_event} \n")
            else:
                print(f" S : \n{e_home} -> {fuzzy_home}")
                print(f"{e_away} -> {fuzzy_away}")
                print(f"datetime : {e_date}")
                print(f"Min Dt : {min_dt_of_event}")
            
            wrong+=1
            print("❌ No close match found.")
        # if corr+wrong+ups != len(merged):
        #     print(f"{corr}+{wrong}+{ups} != {len(merged)}\n")
        #     print(row)
    if return_counts :
        return corr,wrong,ups
    return pd.DataFrame(merged),corr,wrong,ups
# results_df = pd.DataFrame(results)

### For counting Numbers Only

In [14]:
data_collection = []
for season,event_path,shot_path in map_example:
    e_df = pd.read_excel(event_path).sort_values(['home_team','away_team','datetime'])
    # s_df = pd.read_excel(shot_path).sort_values(['home_team','away_team','datetime'])
    s_df = pd.concat([pd.read_excel(x) for x in shot_path.split("__||__")])
    event_teams = list(set(e_df['home_team'].to_list()+e_df['away_team'].to_list()))
    shot_teams = list(set(s_df['home_team'].to_list()+s_df['away_team'].to_list()))

    temp = {}
    temp['Season'] = f"{season.competition.competition_name} | {season.name_fotmob}"
    temp['Total Events'] = e_df.shape[0] if e_df.shape[0] else -1
    temp['Total Shot'] = s_df.shape[0] if s_df.shape[0] else -1
    temp['Correctly Mapped'],temp['Wrong Mapped'],temp['Multiple Mapped'] = match_and_count_for_season(e_df,s_df,event_teams,shot_teams,8,return_counts=True)
    temp['Issues'] =  min(temp['Total Events'], temp['Total Events']) - temp['Correctly Mapped']
    temp['Match'] = temp['Total Events']==temp['Total Events']
    temp['%'] = round((temp['Issues'])/min(temp['Total Events'], temp['Total Events']))*100
    # temp['%'] = round((temp['Issues']/temp['Total Events']))
    data_collection.append(temp)


fakk, there are more ... do something kent saab
fakk, there are more ... do something kent saab
fakk, there are more ... do something kent saab
fakk, there are more ... do something kent saab
fakk, there are more ... do something kent saab
fakk, there are more ... do something kent saab
fakk, there are more ... do something kent saab
fakk, there are more ... do something kent saab
fakk, there are more ... do something kent saab
fakk, there are more ... do something kent saab
 S : 
Al Batin Club -> {'Al Batin': 100.0}
Al Ettifaq Club -> {'Al-Ettifaq': 100.0}
datetime : 2019-01-11 18:00:00
Min Dt : 2018-08-30 15:40:00
❌ No close match found.
 S : 
Al Ettifaq Club -> {'Al-Ettifaq': 100.0}
Al Batin Club -> {'Al Batin': 100.0}
datetime : 2018-09-13 20:55:00
Min Dt : 2018-08-30 15:40:00
❌ No close match found.
fakk, there are more ... do something kent saab
fakk, there are more ... do something kent saab
 S : 
AA Argentinos Juniors -> {'Argentinos Juniors': 100.0, 'Boca Juniors': 81.81818181

In [15]:
pd.DataFrame(data_collection).to_excel("Data Collection 5.xlsx",index=False)

#### For Getting merged Dfs

In [16]:
merged_data_collection = []
BASE_META_DATA="D:/Merged_Items"
err,done=0,0
for season,event_path,shot_path in map_example:
    print(f"Working on {done+err+1}/{len(map_example)} | Completed : {done} | Failed {err}")
    try :
        e_df = pd.read_excel(event_path).sort_values(['home_team','away_team','datetime'])
        s_df = pd.concat([pd.read_excel(x) for x in shot_path.split("__||__")])
        # s_df = pd.read_excel(shot_path).sort_values(['home_team','away_team','datetime'])
        
        event_teams = list(set(e_df['home_team'].to_list()+e_df['away_team'].to_list()))
        shot_teams = list(set(s_df['home_team'].to_list()+s_df['away_team'].to_list()))
    
        conf = season.competition.confederation
        country = season.competition.country
        name = season.competition.competition_name
        df,corr,wrong,ups = match_and_count_for_season(e_df,s_df,event_teams,shot_teams,8,return_counts=False)
        print("Merging Done !!")
        os.makedirs(BASE_META_DATA, exist_ok=True)
        os.makedirs(f"{BASE_META_DATA}/{conf}", exist_ok=True)
        os.makedirs(f"{BASE_META_DATA}/{conf}/{country}", exist_ok=True)
        merge_path = f"{BASE_META_DATA}/{conf}/{country}/{name.replace(' ','_')}__{season.name_fotmob.replace('/','_').replace(' - ','_').replace(' ','_').replace('-','_')}__Merged.xlsx"
        df.to_excel(merge_path,index=False)
        
        temp = {}
        temp['competition'] = f"{season.competition.competition_name}"
        temp['season'] = f"{season.name_fotmob}"
        
        
        temp['events_count'] = e_df.shape[0]
        temp['shot_count'] = s_df.shape[0]
        temp['merged_count'] = s_df.shape[0]
    
        temp['events_path'] = event_path
        temp['shot_path'] = shot_path
        temp['merged_path'] = merge_path
        merged_data_collection.append(temp)
        print(f"Completed '{season.competition.competition_name} | {season.name_fotmob}'")
        done+=1
    except Exception as e :
        print(f"Failed '{season.competition.competition_name} | {season.name_fotmob}'")
        print(e)
        err+=1

Working on 1/199 | Completed : 0 | Failed 0
Merging Done !!
Completed 'A-League | 2024/2025'
Working on 2/199 | Completed : 1 | Failed 0
Merging Done !!
Completed 'A-League | 2023/2024'
Working on 3/199 | Completed : 2 | Failed 0
Merging Done !!
Completed 'A-League | 2022/2023'
Working on 4/199 | Completed : 3 | Failed 0
Merging Done !!
Completed 'A-League | 2021/2022'
Working on 5/199 | Completed : 4 | Failed 0
Merging Done !!
Completed 'A-League | 2020/2021'
Working on 6/199 | Completed : 5 | Failed 0
Merging Done !!
Completed 'A-League | 2019/2020'
Working on 7/199 | Completed : 6 | Failed 0
Merging Done !!
Completed 'A-League | 2018/2019'
Working on 8/199 | Completed : 7 | Failed 0
Merging Done !!
Completed 'A-League | 2017/2018'
Working on 9/199 | Completed : 8 | Failed 0
Merging Done !!
Completed 'Indian Super League | 2024/2025'
Working on 10/199 | Completed : 9 | Failed 0
Merging Done !!
Completed 'Indian Super League | 2023/2024'
Working on 11/199 | Completed : 10 | Failed 0
M

## Phase 1 : Add whatever is there

In [ ]:
safe_combo_seasons = safe_df['Season']

non mapped

In [ ]:
# for i,(season,event_path) in enumerate(non_mapped_season,1) :
#     ev_df = pd.read_excel(event_path)
#     for j,row in ev_df.iterrows():
#         print(j)
#         print(row)
#         obj = Game(
#                 season = season,
#                 home_team = row.get('home_team'),
#                 away_team = row.get('away_team'),
#                 match_date = datetime.strptime(str(row.get('datetime')),"%Y-%m-%d %H:%M:%S"),
#                 game_event_url = row.get('event_url')
#         )
#         print(obj.__dict__)
#         break
#     break

In [ ]:
# season = models.ForeignKey(Season, on_delete=models.CASCADE)
# home_team = models.CharField(max_length=50, null=False, blank=False)
# away_team = models.CharField(max_length=60, null=False, blank=False)
# match_date = models.DateTimeField(null=False, blank=False)
# EVENT_STATUS_CHOICES = [
#     ('not_done', 'Not Done'),
#     ('in_progress', 'In Progress'),
#     ('completed', 'Completed'),
#     ('error', 'Error'),
# ]
# event_status = models.CharField(
#     max_length=20,
#     choices=EVENT_STATUS_CHOICES,
#     default='not_done',
#     null=False,
#     blank=False
# )

# game_event_url = models.URLField(null=True,blank=True,default="")
# game_shot_url = models.URLField(null=True,blank=True,default="")

In [ ]:
from leagues.models import Competition,Season
from base_app.helpers import best_fuzzy_match
from games.models import Game
from datetime import datetime
import pandas as pd
import numpy as np
import os


In [18]:
from datetime import timedelta
                               # e_df,s_df,event_teams,shot_teams,mapping_dict,8,return_counts=True
def match_and_count_for_season(e_df,s_df,event_teams,shot_teams,hrs,return_counts=False):
    max_diff_hours = hrs
    results = []
    merged = []
    corr = 0
    wrong = 0
    ups = 0
    not_req = 0
    min_dt_of_event = s_df['datetime'].min()
    for i, row in e_df.iterrows():
        e_home = row['home_team']
        e_away = row['away_team']
        e_date = row['datetime']

        if (e_date-min_dt_of_event).total_seconds()/3600 < -8 :
            not_req+=1
            continue
        fuzzy_home = best_fuzzy_match(e_home, shot_teams,threshold_multi=85)
        fuzzy_away = best_fuzzy_match(e_away, shot_teams,threshold_multi=85)
        
        # Get top fuzzy keys (most probable names)
        possible_homes = list(fuzzy_home.keys())
        possible_aways = list(fuzzy_away.keys())
        
        # Check all possible home-away combinations in df
        mask = (
            (s_df['home_team'].isin(possible_homes)) &
            (s_df['away_team'].isin(possible_aways)) &
            (abs(s_df['datetime'] - e_date) <= timedelta(hours=max_diff_hours))
        )
        
        matches = s_df[mask]
        if matches.shape[0] >1:
            print("fakk, there are more ... do something kent saab")
            # print(f"\n{e_home} -> {fuzzy_home}")
            # print(f"{e_away} -> {fuzzy_away}")
            # print(f"datetime : {e_date} \n\n")
            
            # ups+=1
            fuzzy_home = best_fuzzy_match(e_home, shot_teams,threshold_multi=78)
            fuzzy_away = best_fuzzy_match(e_away, shot_teams,threshold_multi=78)
            
            # Get top fuzzy keys (most probable names)
            possible_homes = list(fuzzy_home.keys())
            possible_aways = list(fuzzy_away.keys())
            
            # Check all possible home-away combinations in df
            mask = (
                (s_df['home_team'].isin(possible_homes)) &
                (s_df['away_team'].isin(possible_aways)) &
                (abs(s_df['datetime'] - e_date) <= timedelta(hours=max_diff_hours))
            )
            
            matches = s_df[mask]
            if matches.shape[0] > 1 :
                print("Fadd lo bhai hun dooji waar!!!!!!!!!!")
                print(f"\n{e_home} -> {fuzzy_home}")
                print(f"{e_away} -> {fuzzy_away}")
                print(f"datetime : {e_date} \n\n")
                ups+=1
                
        # print(f"Found {matches.shape[0]} matched for the fuzzy map combinations")
        # print(f"\n{e_home} -> {fuzzy_home}")
        # print(f"{e_away} -> {fuzzy_away}")
        # print(f"datetime : {e_date}")
        if not matches.empty:
            correct_match = matches.iloc[0]
                    
            temp = {}
            temp['e_home_team'] = e_home
            temp['s_home_team'] = correct_match['home_team']
            
            temp['e_away_team'] = e_away
            temp['s_away_team'] = correct_match['away_team']
        
            temp['e_datetime'] = e_date
            temp['s_datetime'] = correct_match['datetime']
        
            temp['event_url'] = row['event_url']
            temp['shot_url'] = correct_match['shot_url']
            merged.append(temp) 
            corr+=1
        else:
            # print("\n Adding Blanks \n")
            temp = {}
            # temp['e_home_team'] = e_home
            # temp['s_home_team'] = "NA"
            
            # temp['e_away_team'] = e_away
            # temp['s_away_team'] = "NA"
        
            # temp['e_datetime'] = e_date
            # temp['s_datetime'] = "NA"
        
            # temp['event_url'] = row['event_url']
            # temp['shot_url'] = "NA"
            # merged.append(temp) 
            if fuzzy_home == {} or fuzzy_away == {} :
            
                # print(row)
                # print()
                print(f"\nF : {e_home} -> {fuzzy_home}")
                print(f"{e_away} -> {fuzzy_away}")
                print(f"datetime : {e_date}")
                print(f"Diff : {e_date-min_dt_of_event} \n")
            else:
                print(f" S : \n{e_home} -> {fuzzy_home}")
                print(f"{e_away} -> {fuzzy_away}")
                print(f"datetime : {e_date}")
                print(f"Min Dt : {min_dt_of_event}")
            
            wrong+=1
            print("❌ No close match found.")
        # if corr+wrong+ups != len(merged):
        #     print(f"{corr}+{wrong}+{ups} != {len(merged)}\n")
        #     print(row)
    if return_counts :
        return corr,wrong,ups
    return pd.DataFrame(merged),corr,wrong,ups
# results_df = pd.DataFrame(results)

In [19]:
merged_data_collection = []
BASE_META_DATA="D:/Merged_Items"
err,done=0,0

In [20]:
Map_example = [x for x in map_example if 'Liga MX' in x[0].competition.competition_name]
Map_example

[(<Season: Liga MX - 2024/2025 - FM(2024/2025 - Apertura)>,
  'D:\\MetaData\\CONCACAF\\Mexico\\Liga_MX_2024_2025_events.xlsx',
  'D:\\MetaData-shot\\CONCACAF\\Mexico\\Liga_MX_2024_2025_Apertura_shots.xlsx__||__D:\\MetaData-shot\\CONCACAF\\Mexico\\Liga_MX_2024_2025_Clausura_shots.xlsx'),
 (<Season: Liga MX - 2023/2024 - FM(2023/2024 - Apertura)>,
  'D:\\MetaData\\CONCACAF\\Mexico\\Liga_MX_2023_2024_events.xlsx',
  'D:\\MetaData-shot\\CONCACAF\\Mexico\\Liga_MX_2023_2024_Apertura_shots.xlsx__||__D:\\MetaData-shot\\CONCACAF\\Mexico\\Liga_MX_2023_2024_Clausura_shots.xlsx'),
 (<Season: Liga MX - 2022/2023 - FM(2022/2023 - Apertura)>,
  'D:\\MetaData\\CONCACAF\\Mexico\\Liga_MX_2022_2023_events.xlsx',
  'D:\\MetaData-shot\\CONCACAF\\Mexico\\Liga_MX_2022_2023_Apertura_shots.xlsx__||__D:\\MetaData-shot\\CONCACAF\\Mexico\\Liga_MX_2022_2023_Clausura_shots.xlsx'),
 (<Season: Liga MX - 2021/2022 - FM(2021/2022 - Apertura)>,
  'D:\\MetaData\\CONCACAF\\Mexico\\Liga_MX_2021_2022_events.xlsx',
  'D:\\Me

In [21]:
done=0
err=0

In [22]:
for season,event_path,shot_path in Map_example[:1]:
    print(f"Working on {done+err+1}/{len(map_example)} | Completed : {done} | Failed {err} | {season.name}")
    try :
        e_df = pd.read_excel(event_path).sort_values(['datetime','home_team','away_team'])
        # s_df = pd.read_excel(shot_path).sort_values(['datetime','home_team','away_team'])
        s_df = pd.concat([pd.read_excel(x) for x in shot_path.split("__||__")])
        print(s_df.shape)
        event_teams = list(set(e_df['home_team'].to_list()+e_df['away_team'].to_list()))
        shot_teams = list(set(s_df['home_team'].to_list()+s_df['away_team'].to_list()))
    
        conf = season.competition.confederation
        country = season.competition.country
        name = season.competition.competition_name
        df,corr,wrong,ups = match_and_count_for_season(e_df,s_df,event_teams,shot_teams,11,return_counts=False)
        print("Merging Done !!")
        print(f" df : {df.shape[0]}/{e_df.shape[0]}/{s_df.shape[0]} | corr : {corr} | wrong : {wrong} | ups : {ups}")
        continue
        os.makedirs(BASE_META_DATA, exist_ok=True)
        os.makedirs(f"{BASE_META_DATA}/{conf}", exist_ok=True)
        os.makedirs(f"{BASE_META_DATA}/{conf}/{country}", exist_ok=True)
        merge_path = f"{BASE_META_DATA}/{conf}/{country}/{name.replace(' ','_')}__{season.name_fotmob.replace('/','_').replace(' - ','_').replace(' ','_').replace('-','_')}__Merged.xlsx"
        df.to_excel(merge_path,index=False)
        
        temp = {}
        temp['competition'] = f"{season.competition.competition_name}"
        temp['season'] = f"{season.name_fotmob}"
        
        
        temp['events_count'] = e_df.shape[0]
        temp['shot_count'] = s_df.shape[0]
        temp['merged_count'] = s_df.shape[0]
    
        temp['events_path'] = event_path
        temp['shot_path'] = shot_path
        temp['merged_path'] = merge_path
        merged_data_collection.append(temp)
        print(f"Completed '{season.competition.competition_name} | {season.name_fotmob}'")
        done+=1
    except Exception as e :
        print(f"Failed '{season.competition.competition_name} | {season.name_fotmob}'")
        print(e)
        err+=1

Working on 1/199 | Completed : 0 | Failed 0 | 2024/2025
(340, 6)

F : CD Guadalajara -> {}
Deportivo Toluca FC -> {'Toluca': 100.0}
datetime : 2024-07-07 06:30:00
Diff : 1 days 07:45:00 

❌ No close match found.

F : Club Universidad Nacional -> {}
Club León -> {'Leon': 100.0}
datetime : 2024-07-07 23:30:00
Diff : 2 days 00:45:00 

❌ No close match found.

F : Club Tijuana Xoloitzcuintles de Caliente -> {'Tijuana': 100.0}
CD Guadalajara -> {}
datetime : 2024-07-13 08:30:00
Diff : 7 days 09:45:00 

❌ No close match found.

F : Club Santos Laguna -> {'Santos Laguna': 100.0}
Club Universidad Nacional -> {}
datetime : 2024-07-14 06:55:00
Diff : 8 days 08:10:00 

❌ No close match found.

F : Querétaro FC -> {'Queretaro FC': 100.0}
CD Guadalajara -> {}
datetime : 2024-07-17 06:30:00
Diff : 11 days 07:45:00 

❌ No close match found.

F : FC Juárez -> {'FC Juarez': 100.0}
Club Universidad Nacional -> {}
datetime : 2024-07-18 06:36:00
Diff : 12 days 07:51:00 

❌ No close match found.

F : CD Gu

In [23]:
home = "CD Guadalajara"
away = "Deportivo Toluca FC"
e_date = datetime.strptime("2024-07-07 06:30:00","%Y-%m-%d %H:%M:%S")

In [25]:
fuzzy_home = best_fuzzy_match(home, shot_teams)
fuzzy_away = best_fuzzy_match(away, shot_teams)

# Get top fuzzy keys (most probable names)
possible_homes = list(fuzzy_home.keys())
possible_aways = list(fuzzy_away.keys())

# Check all possible home-away combinations in df
mask = (
    # (s_df['home_team'].isin(possible_homes))
    (s_df['away_team'].isin(possible_aways))
    & (abs(s_df['datetime'] - e_date) <= timedelta(hours=20))
)

matches = s_df[mask]
# print(abs(list(matches['datetime'])[0] - e_date),e_date)
matches

,home_team,home_score,away_team,away_score,datetime,shot_url
4,Chivas,0,Toluca,0,2024-07-07 01:00:00,https://www.fotmob.com/matches/toluca-vs-chiva...


In [15]:
from base_app.TEAM_MAP import TEAM_MAPPING
from rapidfuzz import fuzz
from unidecode import unidecode
import re# Internal: normalize a team name (accents, lowercase, separators → spaces)
def _normalize_team(name: str) -> str:
    name = unidecode(name).lower().strip()
    name = re.sub(r'[^a-z0-9]+', ' ', name)
    return name
# Internal: compute acronym from team name
def _acronym(name: str) -> str:
    tokens = name.split()
    return "".join(t[0] for t in tokens if t)


In [24]:
def dummy(team_name, choices, threshold_single=90, threshold_multi=75):
    q_norm = _normalize_team(team_name)
    q_tokens = q_norm.split()
    single_word = len(q_tokens) == 1
    q_acr = _acronym(q_norm)
    
    # Normalize mapping bidirectionally
    normalized_mapping = {}
    for k, v in TEAM_MAPPING.items():
        # k_norm = _normalize_team(k)
        # v_norm = _normalize_team(v)
        # normalized_mapping[k_norm] = v_norm
        # normalized_mapping[v_norm] = k_norm  # bidirectional
        k_norm = _normalize_team(k)
        # v_norm = [_normalize_team(v)
        normalized_mapping[k_norm] = [_normalize_team(x) for x in v]
        # normalized_mapping[v_norm] = k_norm  # bidirectional
        for x in v :
            normalized_mapping[_normalize_team(x)] = k_norm
    results = {}
    for choice in choices:
        c_norm = _normalize_team(choice)
        c_acr = _acronym(c_norm)
        score = 0
        # Mapping check (bidirectional)
        for map_key, map_val in normalized_mapping.items():
            if (q_norm == map_key and c_norm == map_val) or (q_norm == map_val and c_norm == map_key):
                score = 101
                break
            if c_norm in normalized_mapping.get(q_norm,[]):
                print(f"{q_norm} : {q_acr} | {normalized_mapping.get(q_norm,[])}")
                print(c_norm in normalized_mapping.get(q_norm,[]),c_acr in normalized_mapping.get(q_norm,[]))
                score=102
                break
        # Regular fuzzy scoring if mapping didn't match
        if score == 0:
            ts = fuzz.token_set_ratio(q_norm, c_norm)
            tsort = fuzz.token_sort_ratio(q_norm, c_norm)
            pr = fuzz.partial_ratio(q_norm, c_norm)
            score = max(ts, tsort, pr)

            # Acronym logic
            acr_score = 0
            if len(q_norm) <= 4 and q_norm.isalpha():
                if q_norm == c_acr:
                    acr_score = 100
            else:
                if q_acr == c_acr and len(c_acr) <= 4:
                    acr_score = 100
            score = max(score, acr_score)

        # Apply thresholds
        if single_word:
            if score >= threshold_single:
                results[choice] = score
        else:
            if score >= threshold_multi:
                results[choice] = score
    # print("results : ",results)
    # print(team_name,"score : ",score)
    
    return dict(sorted(results.items(), key=lambda x: x[1], reverse=True))
# dummy(problem[0],shot_teams)
for x in sorted(event_teams) :
    # dummy(x,shot_teams)
    print(x," | ",dummy(x,shot_teams),"\n")

aek athens fc : aaf | ['aek athens', 'aek']
True False
AEK Athens FC  |  {'AEK Athens': 102} 

afc ajax : aa | ['ajax', 'ajax amsterdam']
True False
AFC Ajax  |  {'Ajax': 102, 'AEK Athens': 100} 

as monaco fc : amf | ['monaco', 'as monaco']
True False
AS Monaco FC  |  {'Monaco': 102} 

AS Roma  |  {'Roma': 100.0} 

bsc young boys : byb | ['young boys']
True False
BSC Young Boys  |  {'Young Boys': 102, 'Lyon': 75.0} 

BV Borussia 09 Dortmund  |  {'Borussia Dortmund': 100.0} 

Club Atlético de Madrid  |  {'Atletico Madrid': 100.0, 'Real Madrid': 84.21052631578947} 

Club Brugge KV  |  {'Club Brugge': 100.0} 

FC Barcelona  |  {'Barcelona': 100.0, 'Lyon': 75.0} 

FC Bayern München  |  {'Bayern München': 100.0} 

FC Dynamo Kyiv  |  {} 

FC Internazionale Milano  |  {'Inter': 100.0} 

FC Porto  |  {'FC Porto': 100.0} 

FC Salzburg  |  {} 

FC Schalke 04  |  {'Schalke 04': 100.0} 

FC Shakhtar Donetsk  |  {'Shakhtar Donetsk': 100.0} 

fc viktoria plzen : fvp | ['viktoria plzen']
True False


In [19]:
problem=["Royal Standard de Liège","PAOK Thessaloniki FC"]

In [24]:
sorted(shot_teams)

['AEK Athens',
 'Ajax',
 'Atletico Madrid',
 'Barcelona',
 'Bayern München',
 'Benfica',
 'Borussia Dortmund',
 'CSKA Moscow',
 'Club Brugge',
 'FC Porto',
 'FK Crvena Zvezda',
 'Galatasaray',
 'Hoffenheim',
 'Inter',
 'Juventus',
 'Liverpool',
 'Lokomotiv Moscow',
 'Lyon',
 'Manchester City',
 'Manchester United',
 'Monaco',
 'Napoli',
 'PSV Eindhoven',
 'Paris Saint-Germain',
 'Real Madrid',
 'Roma',
 'Schalke 04',
 'Shakhtar Donetsk',
 'Tottenham Hotspur',
 'Valencia',
 'Viktoria Plzen',
 'Young Boys']